In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import matplotlib
import astropy.units as u
import astropy.coordinates as coord
from astroquery.vizier import Vizier
%matplotlib inline
from astropy.io import fits
from astroquery.simbad import Simbad
import matplotlib
import aplpy
from astroquery.sdss import SDSS
from astropy.coordinates import Angle
import time

/Users/niallbradley/anaconda/lib/python2.7/site-packages/astroquery/sdss/__init__.py:29: UserWarning: Experimental: SDSS has not yet been refactored to have its API match the rest of astroquery (but it's nearly there).
  warnings.warn("Experimental: SDSS has not yet been refactored to have its API "


In [2]:
def Corrolate_Catalogues(RA,DE,Radius,First_Catalogue_Name,Second_Catalogue_Name,First_Flux,Second_Flux,First_error_RA,First_error_DE,Second_error_RA,Second_error_DE):
    #Getting data from first catalogue
    First_Catalogue_Data = Vizier(row_limit= -1,catalog=First_Catalogue_Name,columns=['ALL', '_RAJ2000', '_DEJ2000',First_Flux,First_error_RA,First_error_DE,]).query_region(coord.SkyCoord(ra=RA, dec=DE,unit=(u.deg, u.deg),frame='icrs'),radius=Angle(Radius, "deg"))[0]
    #Getting data from second catalogue
    Second_Catalogue_Data = Vizier(row_limit= -1,catalog=Second_Catalogue_Name,columns=['ALL', '_RAJ2000', '_DEJ2000',Second_Flux,Second_error_RA,Second_error_DE,]).query_region(coord.SkyCoord(ra=RA, dec=DE,unit=(u.deg, u.deg),frame='icrs'),radius=Angle(Radius, "deg"))[0]
    #First Catalogue:

    #Making array of coords

    RA_First=[]
    for value in First_Catalogue_Data["_RAJ2000"]:
        RA_First.append(value)
    #print len(RA_First)
    DE_First=[]
    for value in First_Catalogue_Data["_DEJ2000"]:
        DE_First.append(value)
    #print len(DE_First)

    #Flux Array:

    Lum_First=[]
    for value in First_Catalogue_Data[First_Flux]:
        Lum_First.append(value)
    #print Lum_First

    #Error array:

    RA_First_e=[]
    for value in First_Catalogue_Data[First_error_RA]:
        RA_First_e.append(value/3600)
    #print RA_First_e
    DE_First_e=[]
    for value in First_Catalogue_Data[First_error_DE]:
        DE_First_e.append(value/3600)
    #print DE_First_e

    #Second Catalogue

    #Making array of coords

    RA_Second=[]
    for value in Second_Catalogue_Data["_RAJ2000"]:
        RA_Second.append(value)
    #print len(RA_Second)
    DE_Second=[]
    for value in Second_Catalogue_Data["_DEJ2000"]:
        DE_Second.append(value)
    #print DE_Second

    #Flux Array:

    Lum_Second=[]
    for value in Second_Catalogue_Data[Second_Flux]:
        Lum_Second.append(value)
    #print Lum_Second

    #Error array:

    RA_Second_e=[]
    for value in Second_Catalogue_Data[Second_error_RA]:
        RA_Second_e.append(value/3600)
    #print len(RA_Second_e)
    DE_Second_e=[]
    for value in Second_Catalogue_Data[Second_error_DE]:
        DE_Second_e.append(value/3600)
    #print DE_Second_e
    
    Lum_First_Relevant=[]
    Lum_Second_Relevant=[]
    RA_First_Relevant=[]
    RA_Second_Relevant=[]
    DE_First_Relevant=[]
    DE_Second_Relevant=[]
    #Corrolated_Ss=[]
    f=0
    s=0
    n=0
    m=1
    corr=1
    #go through both data sets
    while f<len(RA_First):
        if s==len(RA_Second):
            f=f+1
            s=0
        #if coordinates within error range then..
        elif (RA_First_e[f]-RA_Second_e[s])**2 <= (RA_First[f]-RA_Second[s])**2+(DE_First[f]-DE_Second[s])**2 and (RA_First[f]-RA_Second[s])**2+(DE_First[f]-DE_Second[s])**2 <=(RA_First_e[f]+RA_Second_e[s])**2:
            #Making arrays of flux of stars in common (Divided by 1000 for units)
            Lum_First_Relevant.append((Lum_First[f]*.001))
            Lum_Second_Relevant.append(Lum_Second[s]*.001)
        
            RA_First_Relevant.append(RA_First[f])
            RA_Second_Relevant.append(RA_Second[s])
        
            DE_First_Relevant.append(DE_First[f])
            DE_Second_Relevant.append(DE_Second[s])
            #Corrolated_Ss.append(s)
            corr=corr+1
            m=m+1
            f=f+1
            s=0
        else:
            s=s+1
        n=n+1
   
    
    print "Number of iterations:",n
    print "Number of stars corrolated:",corr
    
    #Plotting coords of two arrays to compare


    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.scatter(RA_First, DE_First, s=10, c='b', marker="s", label=First_Catalogue_Name)
    ax1.scatter(RA_Second,DE_Second, s=10, c='r', marker="o", label=Second_Catalogue_Name)
    plt.legend(loc='upper left');
    plt.show()
    
    #Plotting Luminosities (unit Jy, log scale)

    if corr>0:
        fig = plt.figure()
        ax1 = fig.add_subplot(111)
        ax1.scatter(Lum_First_Relevant, Lum_Second_Relevant)
        plt.xlabel(First_Catalogue_Name)
        plt.ylabel(Second_Catalogue_Name)
        plt.xscale('log')
        plt.yscale('log')
        plt.xlim(0.0001,10)
        plt.ylim(0,10)
        plt.show()
    
    Lum_diff=[]
    x=0
    n=0
    j=0
    Max_Diff=0
    while n<corr-1:
        x=(abs(Lum_First_Relevant[n]-Lum_Second_Relevant[n]))/Lum_First_Relevant[n]
        if x>j:
            Max_Diff=n
            j=x
        n=n+1
    print RA_First_Relevant[Max_Diff],RA_Second_Relevant[Max_Diff]
    
    pos=coord.SkyCoord(RA_First_Relevant[Max_Diff]*u.deg, DE_First_Relevant[Max_Diff]*u.deg)
    ra=np.array([RA_First_Relevant[Max_Diff]])
    de=np.array([DE_First_Relevant[Max_Diff]])
    radius=np.array([50.0])

    images = SDSS.get_images(pos)
    CV = aplpy.FITSFigure(images[0])
    CV.show_grayscale()
    CV.add_grid()
    #CV.show_markers(ra, de, edgecolor='green', facecolor='none',marker='o', s=10, alpha=0.5)

    x_pix, y_pix = CV.world2pixel(ra,de)


    plt.scatter(x_pix, y_pix, s=100, c='red', alpha=0.4)
    plt.show()
    
    


In [3]:
Corrolate_Catalogues(165.0,50.0,2,'LOTSS','TGSS','FTot','Stotal','e_RAdegtot','e_DEdegtot','e_RAJ2000','e_DEJ2000')

KeyError: 'FTot'